# Kubeflow pipelines

**Learning Objectives:**
  1. Learn how to deploy a Kubeflow cluster on GCP
  1. Learn how to create a experiment in Kubeflow
  1. Learn how to package you code into a Kubeflow pipeline
  1. Learn how to run a Kubeflow pipeline in a repeatable and traceable way


## Introduction

In this notebook, we will first setup a Kubeflow cluster on GCP.
Then, we will create a Kubeflow experiment and a Kubflow pipeline from our taxifare machine learning code. At last, we will run the pipeline on the Kubeflow cluster, providing us with a reproducible and traceable way to execute machine learning code.

In [1]:
!pip3 install --user kfp --upgrade

### Restart the kernel

After you install the additional packages, you need to restart the notebook kernel so it can find the packages.

### Import libraries and define constants

In [1]:
from os import path

import kfp
import kfp.compiler as compiler
import kfp.components as comp
import kfp.dsl as dsl
import kfp.gcp as gcp
import kfp.notebook

## Setup a Kubeflow cluster on GCP

**TODO 1**

To deploy a [Kubeflow](https://www.kubeflow.org/) cluster
in your GCP project, use the [AI Platform pipelines](https://console.cloud.google.com/ai-platform/pipelines):

1. Go to [AI Platform Pipelines](https://console.cloud.google.com/ai-platform/pipelines) in the GCP Console.
1. Create a new instance
2. Hit "Configure"
3. Check the box "Allow access to the following Cloud APIs"
1. Hit "Create Cluster"
4. Hit "Deploy"

When the cluster is ready, go back to the AI Platform pipelines page and click on "SETTINGS" entry for your cluster.
This will bring up a pop up with code snippets on how to access the cluster 
programmatically. 

Copy the "host" entry and set the "HOST" variable below with that.


In [47]:
HOST = ""  # None  # TODO: fill in the HOST information for the cluster

### Authenticate your KFP cluster with a  Kubernetes secret

If you run pipelines that requires calling any GCP services, you need to set the application default credential to a pipeline step by mounting the proper GCP service account token as a Kubernetes secret.

First point your kubectl current context to your cluster. Go back to your [Kubeflow cluster dashboard](https://console.cloud.google.com/ai-platform/pipelines/clusters) or navigate to `Navigation menu > AI Platform > Pipelines` and look to see the cluster name, zone and namespace for the pipeline you deployed above. It's likely called `cluster-1` if this is the first AI Pipelines you've created. 

In [8]:
PROJECT = !gcloud config get-value project  # noqa: E999
PROJECT = PROJECT[0]
BUCKET = PROJECT  # change if needed
CLUSTER = "cluster-1"  # change if needed
ZONE = "us-central1-a"  # change if needed
NAMESPACE = "default"  # change if needed

%env PROJECT=$PROJECT
%env CLUSTER=$CLUSTER
%env ZONE=$ZONE
%env NAMESPACE=$NAMESPACE

env: PROJECT=dsparing-sandbox
env: CLUSTER=cluster-1
env: ZONE=us-central1-a
env: NAMESPACE=default


In [40]:
# Configure kubectl to connect with the cluster
!gcloud container clusters get-credentials "$CLUSTER" --zone "$ZONE" --project "$PROJECT"

Fetching cluster endpoint and auth data.
kubeconfig entry generated for cluster-1.


We'll create a service account called `kfpdemo` with the necessary IAM permissions for our cluster secret. We'll give this service account permissions for any GCP services it might need. This `taxifare` pipeline needs access to Cloud Storage, so we'll give it the `storage.admin` role and `ml.admin`. Open a Cloud Shell and copy/paste this code in the terminal there.

```bash
PROJECT=$(gcloud config get-value project)

# Create service account
gcloud iam service-accounts create kfpdemo \
  --display-name kfpdemo --project $PROJECT

# Grant permissions to the service account by binding roles
gcloud projects add-iam-policy-binding $PROJECT \
    --member=serviceAccount:kfpdemo@$PROJECT.iam.gserviceaccount.com \
    --role=roles/storage.admin
    
gcloud projects add-iam-policy-binding $PROJECT \
    --member=serviceAccount:kfpdemo@$PROJECT.iam.gserviceaccount.com \
    --role=roles/ml.admin    
```

Then, we'll create and download a key for this service account and store the service account credential as a Kubernetes secret called `user-gcp-sa` in the cluster.

In [ ]:
%%bash
gcloud iam service-accounts keys create application_default_credentials.json \
  --iam-account kfpdemo@$PROJECT.iam.gserviceaccount.com

In [10]:
# Check that the key was downloaded correctly.
!ls application_default_credentials.json

application_default_credentials.json


In [12]:
# Create a k8s secret. If already exists, override.
!kubectl create secret generic user-gcp-sa \
  --from-file=user-gcp-sa.json=application_default_credentials.json \
  -n $NAMESPACE --dry-run=client -o yaml  |  kubectl apply -f -

secret/user-gcp-sa configured


## Create an experiment

**TODO 2**

We will start by creating a Kubeflow client to pilot the Kubeflow cluster:

In [48]:
client = kfp.Client(host=HOST)

Let's look at the experiments that are running on this cluster. Since you just launched it, you should see only a single "Default" experiment:

In [49]:
client.list_experiments()

{'experiments': [{'created_at': datetime.datetime(2020, 11, 17, 19, 13, 33, tzinfo=tzlocal()),
                  'description': 'All runs created without specifying an '
                                 'experiment will be grouped here.',
                  'id': 'f242234d-b1fd-4151-beec-d2f24c049996',
                  'name': 'Default',
                  'resource_references': None,
                  'storage_state': 'STORAGESTATE_AVAILABLE'},
                 {'created_at': datetime.datetime(2020, 11, 17, 19, 17, 51, tzinfo=tzlocal()),
                  'description': None,
                  'id': '437e8233-5fe3-40bb-8c76-dbd9e1d18fef',
                  'name': 'taxifare',
                  'resource_references': None,
                  'storage_state': 'STORAGESTATE_AVAILABLE'}],
 'next_page_token': None,
 'total_size': 2}

Now let's create a 'taxifare' experiment where we could look at all the various runs of our taxifare pipeline:

In [6]:
exp = client.create_experiment(name="taxifare")

Let's make sure the experiment has been created correctly:

In [50]:
client.list_experiments()

{'experiments': [{'created_at': datetime.datetime(2020, 11, 17, 19, 13, 33, tzinfo=tzlocal()),
                  'description': 'All runs created without specifying an '
                                 'experiment will be grouped here.',
                  'id': 'f242234d-b1fd-4151-beec-d2f24c049996',
                  'name': 'Default',
                  'resource_references': None,
                  'storage_state': 'STORAGESTATE_AVAILABLE'},
                 {'created_at': datetime.datetime(2020, 11, 17, 19, 17, 51, tzinfo=tzlocal()),
                  'description': None,
                  'id': '437e8233-5fe3-40bb-8c76-dbd9e1d18fef',
                  'name': 'taxifare',
                  'resource_references': None,
                  'storage_state': 'STORAGESTATE_AVAILABLE'}],
 'next_page_token': None,
 'total_size': 2}

## Packaging your code into Kubeflow components

We have packaged our taxifare ml pipeline into three components:
* `./components/bq2gcs` that creates the training and evaluation data from BigQuery and exports it to GCS
* `./components/trainjob` that launches the training container on AI-platform and exports the model
* `./components/deploymodel` that deploys the trained model to AI-platform as a REST API

Each of these components has been wrapped into a Docker container, in the same way we did with the taxifare training code in the previous lab.

If you inspect the code in these folders, you'll notice that the `main.py` or `main.sh` files contain the code we previously executed in the notebooks (loading the data to GCS from BQ, or launching a training job to AI-platform, etc.). The last line in the `Dockerfile` tells you that these files are executed when the container is run. 
So we just packaged our ml code into light container images for reproducibility. 

We have made it simple for you to build the container images and push them to the Google Cloud image registry gcr.io in your project:

In [17]:
# Builds the taxifare trainer container in case you skipped the optional part
# of lab 1
!taxifare/scripts/build.sh

Sending build context to Docker daemon  113.2kB
Step 1/4 : FROM gcr.io/deeplearning-platform-release/tf2-cpu
latest: Pulling from deeplearning-platform-release/tf2-cpu

57c49d0f: Pulling fs layer 
40447d26: Pulling fs layer 
2f862619: Pulling fs layer 
2764011e: Pulling fs layer 
4e2c5db1: Pulling fs layer 
52976526: Pulling fs layer 
ff203efe: Pulling fs layer 
d74dd489: Pulling fs layer 
11aef694: Pulling fs layer 
5e47c0d1: Pulling fs layer 
e7716976: Pulling fs layer 
37ccffa2: Pulling fs layer 
4903c042: Pulling fs layer 
68d8f6c7: Pulling fs layer 
304a9c74: Pulling fs layer 
ac48815d: Pulling fs layer 
afb5635c: Pulling fs layer 
55a11a1b: Pulling fs layer 
Digest: sha256:1766be1e60470066cce26bd50acc760a7ba12061519e2da4d093b81e870774a32KDownloading  699.6MB/1.998GB
Status: Downloaded newer image for gcr.io/deeplearning-platform-release/tf2-cpu:latest
 ---> 78f89853a227
Step 2/4 : COPY . /code
 ---> 8bd13b56ad6e
Step 3/4 : WORKDIR /code
 ---> Running in b9a0f0dbb76b
Removing inte

In [18]:
# Pushes the taxifare trainer container to gcr/io
!taxifare/scripts/push.sh

The push refers to repository [gcr.io/qwiklabs-gcp-00-568a75dfa3e1/taxifare_training_container]

921820ae: Preparing 
20255c0c: Preparing 
25428eb4: Preparing 
225e785e: Preparing 
18e59128: Preparing 
93aa8921: Preparing 
0c4f31bb: Preparing 
b5583a70: Preparing 
ce1b9bf5: Preparing 
397c9263: Preparing 
5d53f509: Preparing 
2c0fc305: Preparing 
2aeddb5e: Preparing 
19df8fce: Preparing 
88ee89e7: Preparing 
8f5e88f9: Preparing 
a40d1ffb: Preparing 
4df0ad6c: Preparing 
df553184: Preparing 
02706667: Layer already exists ning-platform-release/tf2-cpu latest: digest: sha256:0a2d0d43957a3fe43df420ca980461de36a09b82c6d711fc163370c6b0507ce7 size: 4505


In [19]:
# Builds the KF component containers and push them to gcr/io
!cd pipelines && make components

make[1]: Entering directory '/home/jupyter/asl-ml-immersion/notebooks/building_production_ml_systems/solutions/pipelines/components/bq2gcs'
rm: cannot remove './venv': No such file or directory
rm: cannot remove './component.yaml': No such file or directory
OK
Sending build context to Docker daemon   16.9kB
Step 1/6 : FROM google/cloud-sdk:latest
latest: Pulling from google/cloud-sdk

d3e4f7b0: Pulling fs layer 
81c9382a: Pulling fs layer 
b1e4ddb7: Pulling fs layer 
05461bdc: Pulling fs layer 
Digest: sha256:12e3701513753324519dcca0cd64c7fd23b00621d08aa91cdbcb2b67c7a27f36[5A
Status: Downloaded newer image for google/cloud-sdk:latest
 ---> 7d28e60605c4
Step 2/6 : RUN apt-get update &&     apt-get install --yes python3-pip
 ---> Running in 382d6f97ee56
Hit:1 http://security.debian.org/debian-security buster/updates InRelease
Hit:2 http://deb.debian.org/debian buster InRelease
Hit:3 http://deb.debian.org/debian buster-updates InRelease
Hit:4 https://packages.cloud.google.com/apt cloud-sd

Now that the container images are pushed to the [registry in your project](https://console.cloud.google.com/gcr), we need to create yaml files describing to Kubeflow how to use these containers. It boils down essentially to
* describing what arguments Kubeflow needs to pass to the containers when it runs them
* telling Kubeflow where to fetch the corresponding Docker images

In the cells below, we have three of these "Kubeflow component description files", one for each of our components.

**TODO 3**

**IMPORTANT: Modify the image URI in the cell 
below to reflect that you pushed the images into the gcr.io associated with your project.**

In [34]:
%%writefile bq2gcs.yaml

name: bq2gcs
    
description: |
    This component creates the training and
    validation datasets as BiqQuery tables and export
    them into a Google Cloud Storage bucket at
    gs://qwiklabs-gcp-00-568a75dfa3e1/taxifare/data.
        
inputs:
    - {name: Input Bucket , type: String, description: 'GCS directory path.'}

implementation:
    container:
        image: gcr.io/qwiklabs-gcp-00-568a75dfa3e1/taxifare-bq2gcs
        args: ["--bucket", {inputValue: Input Bucket}]

Overwriting bq2gcs.yaml


In [28]:
%%writefile trainjob.yaml

name: trainjob
    
description: |
    This component trains a model to predict that taxi fare in NY.
    It takes as argument a GCS bucket and expects its training and
    eval data to be at gs://<BUCKET>/taxifare/data/ and will export
    the trained model at  gs://<BUCKET>/taxifare/model/.
        
inputs:
    - {name: Input Bucket , type: String, description: 'GCS directory path.'}

implementation:
    container:
        image: gcr.io/qwiklabs-gcp-00-568a75dfa3e1/taxifare-trainjob
        args: [{inputValue: Input Bucket}]

Overwriting trainjob.yaml


In [29]:
%%writefile deploymodel.yaml

name: deploymodel
    
description: |
    This component deploys a trained taxifare model on GCP as taxifare:dnn.
    It takes as argument a GCS bucket and expects the model to deploy 
    to be found at gs://<BUCKET>/taxifare/model/export/savedmodel/
        
inputs:
    - {name: Input Bucket , type: String, description: 'GCS directory path.'}

implementation:
    container:
        image: gcr.io/qwiklabs-gcp-00-568a75dfa3e1/taxifare-deploymodel
        args: [{inputValue: Input Bucket}]

Overwriting deploymodel.yaml


## Create a Kubeflow pipeline

The code below creates a kubeflow pipeline by decorating a regular function with the
`@dsl.pipeline` decorator. Now the arguments of this decorated function will be
the input parameters of the Kubeflow pipeline.

Inside the function, we describe the pipeline by
* loading the yaml component files we created above into a Kubeflow `op`
* specifying the order into which the Kubeflow ops should be run

In [51]:
# TODO 3
PIPELINE_TAR = "taxifare.tar.gz"
BQ2GCS_YAML = "./bq2gcs.yaml"
TRAINJOB_YAML = "./trainjob.yaml"
DEPLOYMODEL_YAML = "./deploymodel.yaml"


@dsl.pipeline(
    name="Taxifare",
    description="Train a ml model to predict the taxi fare in NY",
)
def pipeline(gcs_bucket_name="<bucket where data and model will be exported>"):

    bq2gcs_op = comp.load_component_from_file(BQ2GCS_YAML)
    bq2gcs = bq2gcs_op(
        input_bucket=gcs_bucket_name,
    )


"""
    trainjob_op = comp.load_component_from_file(TRAINJOB_YAML)
    trainjob = trainjob_op(
        input_bucket=gcs_bucket_name,
    )

    deploymodel_op = comp.load_component_from_file(DEPLOYMODEL_YAML)
    deploymodel = deploymodel_op(
        input_bucket=gcs_bucket_name,
    )

    trainjob.after(bq2gcs)
    deploymodel.after(trainjob)
"""

'\n    trainjob_op = comp.load_component_from_file(TRAINJOB_YAML)\n    trainjob = trainjob_op(\n        input_bucket=gcs_bucket_name,\n    )\n\n    deploymodel_op = comp.load_component_from_file(DEPLOYMODEL_YAML)\n    deploymodel = deploymodel_op(\n        input_bucket=gcs_bucket_name,\n    )\n\n    trainjob.after(bq2gcs)\n    deploymodel.after(trainjob)\n'

The pipeline function above is then used by the Kubeflow compiler to create a Kubeflow pipeline artifact that can be either uploaded to the Kubeflow cluster from the UI, or programatically, as we will do below:

In [52]:
compiler.Compiler().compile(pipeline, PIPELINE_TAR)

In [53]:
ls $PIPELINE_TAR

taxifare.tar.gz


If you untar and uzip this pipeline artifact, you'll see that the compiler has transformed the
Python description of the pipeline into yaml description!

Now let's feed Kubeflow with our pipeline and run it using our client:

In [54]:
# TODO 4
run = client.run_pipeline(
    experiment_id=exp.id,
    job_name="taxifare",
    pipeline_package_path="taxifare.tar.gz",
    params={
        "gcs_bucket_name": BUCKET,
    },
)

Have a look at the link to monitor the run. 

Now all the runs are nicely organized under the experiment in the UI, and new runs can be either manually launched or scheduled through the UI in a completely repeatable and traceable way!